In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
from dateutil.relativedelta import relativedelta
import pandas as pd
from tqdm import tqdm
import os

#BigQuery client
project_id = 'eco590'
credentials = service_account.Credentials.from_service_account_file('C:/Users/Tim/OneDrive/Desktop/eco590-0165d7bd383e.json')
client = bigquery.Client(credentials=credentials, project=project_id)

# 16 month time range starting from 3 months prior of june 22
start_date = pd.to_datetime('2022-03-01')  # March 2022
end_date = pd.to_datetime('2023-06-30')    # June 2023

 # Generate a SQL query to select user login, event type, and event date
    # from GitHub archive data for specified event types within a date range
    # it then filters events to include only PushEvent, PullRequestEvent,
    # CreateEvent, ForkEvent, and IssuesEvent
def generate_monthly_query(start_suffix, end_suffix):
    """Generate SQL query to fetch event dates for each event type per user."""
    return f"""
SELECT
    actor.login AS user_login,
    type AS event_type,
    FORMAT_TIMESTAMP('%Y-%m-%d', created_at) AS event_date
FROM
    `githubarchive.month.*`
WHERE
    _TABLE_SUFFIX BETWEEN '{start_suffix}' AND '{end_suffix}'
    AND type IN ('PushEvent', 'PullRequestEvent', 'CreateEvent', 'ForkEvent', 'IssuesEvent')
"""

# Prepare for iteration
date_range = pd.date_range(start_date, end_date, freq='MS') 

# Iterate through each month in the date range
#using tqdm i kept track of progression of retrival process
#also after each month processed i saved it to my file path in case of emergecy with outside errros
for i, current_date in enumerate(tqdm(date_range, desc="Processing")):
    start_suffix = current_date.strftime('%Y%m')
    end_suffix = (current_date + pd.offsets.MonthEnd(1)).strftime('%Y%m')
    
    #Execute query
    monthly_query = generate_monthly_query(start_suffix, end_suffix)
    df = client.query(monthly_query).to_dataframe()
    
    # Check for duplicate user_login values in the current month's DataFrame
    duplicates = df.duplicated(subset='user_login', keep=False)
    print(f"Number of duplicates in {start_suffix}: {duplicates.sum()}")
    
# aggregate the current month's DataFrame
#aggregate the dataframe by 'user_login' to compile event data per user
#for each user(row) this operation does the following
# - 'event_dates': Aggregates all event dates into a list
# - 'event_types': Aggregates all event types into a list
# - 'event_counts': Counts the total number of events
#The result is a dataframe with a row per user, including their aggregated event information

    aggregated_df = df.groupby('user_login').agg(
        event_dates=pd.NamedAgg(column='event_date', aggfunc=lambda x: list(x)),
        event_types=pd.NamedAgg(column='event_type', aggfunc=lambda x: list(x)),
        event_counts=pd.NamedAgg(column='event_type', aggfunc='size')
    ).reset_index()
    
    # Save the aggregated DataFrame to path
    csv_file_path = f'C:/Users/Tim/OneDrive/Desktop/thesis/aggregated_data_{start_suffix}.csv'
    aggregated_df.to_csv(csv_file_path, index=False)

    print(f"Aggregated data for {start_suffix} saved to {csv_file_path}")

In [ ]:
import pandas as pd
import numpy as np
import os

# since i have 16 different csv files one for each month. i now want to join the data and combine it into one
#csv fiels are all under same folder / name just with the months date at the end of it to differeneitate all of them

base_path = "C:/Users/Tim/OneDrive/Desktop/thesis/aggregated_data_"
date_range = pd.date_range(start="2022-03", end="2023-06", freq='M')  # Monthly from March 2022 to June 2023

file_paths = [f"{base_path}{date.strftime('%Y%m')}.csv" for date in date_range]

In [ ]:
unique_users_per_month = []

for file_path in file_paths:
    try:
        df = pd.read_csv(file_path)
        unique_users = set(df['user_login'].unique()) 
        unique_users_per_month.append(unique_users)
    except FileNotFoundError:
        print(f"File not found: {file_path}")

In [ ]:
users_in_all_months = set.intersection(*unique_users_per_month)

In [ ]:
filtered_dfs = []

for file_path in file_paths:
    df = pd.read_csv(file_path)
    filtered_df = df[df['user_login'].isin(users_in_all_months)]
    filtered_dfs.append(filtered_df)

# Concatenate all of the filtered dataframes
final_dataset = pd.concat(filtered_dfs)

In [ ]:
# from this dataset we can see each row contains the unique user_login
final_dataset.head(20)

since there are still duplicates in this dataframe i will use a group dunction to extract adnd aggreagte all duplicate users into one unique row for that given user

In [ ]:
final_dataset['event_dates'] = final_dataset['event_dates'].apply(lambda x: x if isinstance(x, list) else [x])
final_dataset['event_types'] = final_dataset['event_types'].apply(lambda x: x if isinstance(x, list) else [x])

# Perform the aggregation
aggregated_data = final_dataset.groupby('user_login').agg({
    'event_counts': 'sum',  # Summing the event counts for each user across all months
    'event_dates': lambda dates: list(pd.core.common.flatten(dates)),  # Concatenating all event_dates lists for each user
    'event_types': lambda types: list(pd.core.common.flatten(types)),  # Concatenating all event_types lists for each user
}).reset_index()

In [ ]:
#here is the manual loop i ran for all 16 months due to memory error and whatever else error arose from it, so just replaced the date of the file 16 times and ran the rest of code below ill mark where it stopped
df1 = pd.read_csv(r"C:\Users\Tim\OneDrive\Desktop\thesis\bigquerydata\aggregated_data_202203.csv")


In [ ]:
#users_df is a list of users who appeared throughout all 16 months of data, thjis was my way to reduce the sample size and deal with memory issues and time constraints
users_df = pd.read_csv(r'C:\Users\Tim\OneDrive\Desktop\thesis\users_in_all_months.csv')
users_in_all_months = users_df['user_login'].tolist()

# Filtering df1 to only include rows where user_login is in users_in_all_months
df1_filtered = df1[df1['user_login'].isin(users_in_all_months)]

df1_filtered.head()

In [ ]:
import pandas as pd
from datetime import datetime

def process_events_without_literal_eval(df):
    df_copy = df.copy()  # have to create a copy to avoid SettingWithCopyWarning error if i did not
    
    for index, row in df_copy.iterrows():
        new_dates = []
        new_types = []
        event_count_decrement = 0

        for date, event_type in zip(row['event_dates'], row['event_types']):
            # Check if the date is on or after March 22, 2022
            if datetime.strptime(date, '%Y-%m-%d') >= datetime(2022, 3, 22):
                new_dates.append(date)
                new_types.append(event_type)
            else:
                event_count_decrement += 1

        # Update the DataFrame with the modified lists and adjusted counts
        df_copy.at[index, 'event_dates'] = new_dates  # No need to convert to string
        df_copy.at[index, 'event_types'] = new_types
        df_copy.at[index, 'event_counts'] -= event_count_decrement

    return df_copy

# Apply the function
df1_filtered_processed = process_events_without_literal_eval(df1_filtered)

In [ ]:
df1_filtered = df1_filtered_processed.copy()

# Specified event types i have
event_types = ['PullRequestEvent', 'PushEvent', 'IssuesEvent', 'CreateEvent', 'ForkEvent']

# Use apply with lambda functions for each event type to count occurrences
for event in event_types:
    df1_filtered[event] = df1_filtered['event_types'].apply(lambda x: x.count(event))